# Boris pusher verification notebook
Here we validates the implementation of the **Boris particle pusher** using the HDF5 outputs produced by pusher.hpp:
- `uniform_bz.h5` : uniform magnetic field $ \mathbf{B} = B_z \hat{\mathbf{z}}$, with $ \mathbf{E}=0$
- `drift_ey.h5` : uniform electric field (typically $E_y \neq 0$) and magnetic field $B_z$, producing an $ \mathbf{E}\times\mathbf{B}$ drift

I check both **qualitative behavior** (expected shapes) and **quantitative invariants** (energy conservation, constant $v_z$, constant $|\mathbf{v}_\perp|$).

## Test 1 — Uniform $B_z$ (no electric field)
The Boris scheme is time-centered and the magnetic rotation step is energy conserving. 
With $ \mathbf{E}=0$ and uniform $ \mathbf{B}=B_z \hat{\mathbf{z}}$:
- The motion is a gyromotion in the perpendicular plane $(v_x, v_y)$.
- The kinetic energy should be conserved:
  $$
  K(t)=\frac{1}{2}m|\mathbf{v}(t)|^2 = \text{constant}
  $$
- The parallel component is constant:
  $$
  v_z(t)=\text{constant}
  $$
- The perpendicular speed should be constant:
  $$
  |\mathbf{v}_\perp|=\sqrt{v_x^2+v_y^2}=\text{constant}
  $$

In velocity space, the trajectory $(v_x(t), v_y(t))$ should be a circle.



In [ ]:
import os
print(os.getcwd())

In [ ]:
import h5py
import matplotlib.pyplot as plt

# 1. Uniform Bz test
uniform_bz = h5py.File("uniform_bz.h5", "r")
vx = uniform_bz["vx"][:]
vy = uniform_bz["vy"][:]
uniform_bz.close()

plt.figure()
plt.plot(vx, vy, color='magenta')
plt.xlabel("vx")
plt.ylabel("vy")
plt.title("Uniform Bz: Particle trajectory in xy-plane")
plt.axis("equal")  # To see a perfect circle
plt.grid(True)
plt.savefig("Uniform_Bz_trajectory.png")
plt.show()

plt.figure()
plt.plot(t, vx, label="vx")
plt.plot(t, vy, label="vy")
plt.plot(t, vz, label="vz")
plt.xlabel("Time step")
plt.ylabel("Velocity")
plt.title("Uniform Bz: velocity components vs time")
plt.legend()
plt.grid(True)
plt.savefig("Uniform_Bz_velocity_vs_time.png", dpi=200)
plt.show()

In uniform $B_z$ and $E=0$, the magnetic field does no work.
The Boris pusher should therefore conserve kinetic energy.
This is verified by the constancy of $|\mathbf{v}|^2$ over time.

In [ ]:
v2 = vx**2 + vy**2 + vz**2

plt.figure()
plt.plot(v2, color = 'violet')
plt.xlabel("Time step")
plt.ylabel(r"$|v|^2$")
plt.title("Uniform Bz: kinetic energy proxy $|v|^2$ (should be constant)")
plt.grid(True)
plt.show()

## Test 2 — $E_y$ drift and $ \mathbf{E}\times\mathbf{B}$ drift

With a uniform magnetic field $ \mathbf{B}=B_z \hat{\mathbf{z}}$ and an electric field (here typically $E_y \neq 0$):
- The particle exhibits gyromotion plus a **guiding center drift**.
- The expected drift velocity is the $ \mathbf{E}\times\mathbf{B}$ drift:
  $$
  \mathbf{v}_{E\times B} = \frac{\mathbf{E}\times\mathbf{B}}{|\mathbf{B}|^2}
  $$
This drift is independent of charge and mass.

I visualize:
- the velocity-space trajectory $(v_x, v_y)$
- the spatial displacement $x(t)$ (expected to show a net drift over time)


In [ ]:
# 2. Drift Ey test
drift_ey = h5py.File("drift_ey.h5", "r")
vx = drift_ey["vx"][:]
vy = drift_ey["vy"][:]
x  = drift_ey["x"][:]
drift_ey.close()

plt.figure()
plt.plot(vx, vy, color='magenta')
plt.xlabel("vx")
plt.ylabel("vy")
plt.title("Drift Ey: Particle trajectory in xy-plane")
plt.axis("equal")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(x, color='magenta')
plt.xlabel("Step")
plt.ylabel("x-position")
plt.title("Drift Ey: Particle x-position over time")
plt.grid(True)
plt.savefig("Drift_Ey_trajectory.png")
plt.show()

In the presence of a uniform magnetic field $B_z$ and an electric field $E_y$:
- $v_x$ and $v_y$ show cyclotron oscillations around a shifted mean value,
  corresponding to the $\mathbf{E}\times\mathbf{B}$ drift.
- $v_z$ remains constant, since there is no force acting along the $z$ direction.

This behavior confirms that the Boris pusher correctly captures both gyromotion
and guiding-center drift without introducing spurious acceleration.


In [ ]:
plt.figure()
plt.plot(t, vx, label="vx")
plt.plot(t, vy, label="vy")
plt.plot(t, vz, label="vz")
plt.xlabel("Time step")
plt.ylabel("Velocity")
plt.title("Drift Ey: velocity components vs time")
plt.legend()
plt.grid(True)
plt.savefig("Drift_Ey_velocity_components.png")
plt.show()